# 🎬 Studio-Level AI Video Dubber (Colab Edition)

This notebook runs the **Hybrid TTS Pipeline**:
- **English Dubs:** Uses `Chatterbox TTS` (Fast, Accurate)
- **Hindi/Urdu Dubs:** Uses `Coqui XTTS v2` (Voice Cloning, Emotion Transfer)
- **Lipsync:** (Coming Soon)

### Instructions:
1. Run **Step 1** to install everything (takes ~1 min).
2. Run **Step 2** to upload your project source code (zip file).
3. Run **Step 3** to upload your video.
4. Run **Step 4** to start dubbing!

In [ ]:
# @title Step 1: Install Dependencies & Setup Environment
# @markdown This installs Coqui TTS, Whisper, Demucs, and system tools.

print("Installing System Dependencies (eSpeak, FFmpeg)...")
!apt-get update -qq
!apt-get install -y espeak-ng ffmpeg

print("Installing Python Libraries (This may take roughly 1-2 minutes)...")
!pip install -q torch torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q TTS chatterbox-tts openai-whisper demucs deep-translator pydub scipy ffmpeg-python numpy librosa transformers huggingface_hub gradio_client openai

print("✅ Environment Ready!")

In [ ]:
# @title Step 2: Upload Project Code
# @markdown Please zip your local `video dubber studio level` folder (excluding `output` and `.venv`) as `project.zip` and upload it here.

import os
from google.colab import files
import shutil

if os.path.exists("project"):
    shutil.rmtree("project")
os.makedirs("project", exist_ok=True)

print("⬇️ Upload your 'project.zip' file now:")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(".zip"):
        print(f"Extracting {filename}...")
        !unzip -q "{filename}" -d project
        print("✅ Project Extracted to /content/project")
        
        # Fix nested folder structure if user zipped the parent folder
        # If project/video dubber studio level/main.py exists, move it up
        for root, dirs, f in os.walk("project"):
            if "main.py" in f:
                project_root = root
                print(f"Found project root at: {project_root}")
                %cd "{project_root}"
                break
    else:
        print("⚠️ Warning: Uploaded file is not a .zip!")


In [ ]:
# @title Step 3: Upload Video to Dub

print("⬇️ Upload your input video (mp4/mkv):")
uploaded_video = files.upload()

input_video_path = list(uploaded_video.keys())[0]
print(f"✅ Video Ready: {input_video_path}")

In [ ]:
# @title Step 4: Run Dubbing Pipeline
# @markdown Choose your target language.

TARGET_LANGUAGE = "hi" # @param ["hi", "ur", "en", "es", "fr", "ja"]
TRANSLATION_SERVICE = "openrouter" # @param ["mistral", "openrouter", "google"]

# IMPORTANT: Set API Keys here if not in your .env file inside the zip
import os
os.environ["OPENROUTER_API_KEY"] = "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"
os.environ["MISTRAL_API_KEY"] = "i0ICrPypIZ0raJVUky81Vu5DRZhIiP6m"

print(f"🚀 Starting Dubbing Process ({TARGET_LANGUAGE})...")
# Running the main.py from the uploaded project
!python main.py "{os.path.abspath(input_video_path)}" --lang {TARGET_LANGUAGE} --service {TRANSLATION_SERVICE}

print("Process Completed! Check the output folder to download your video.")

In [ ]:
# @title (Optional) specific download 
from google.colab import files
import glob

# Find the output video (assuming it's in an 'output' folder or main dir)
output_files = glob.glob("output/*.mp4") + glob.glob("*.mp4")
# Filter out the input video
output_files = [f for f in output_files if f != input_video_path]

if output_files:
    last_video = max(output_files, key=os.path.getctime)
    print(f"Downloading {last_video}...")
    files.download(last_video)
else:
    print("No output video found yet!")